<a href="https://colab.research.google.com/github/dilutha/Pneumonia_X-tray_Detection/blob/main/Automated_Pneumonia_Detection_from_Chest_X_Ray_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow numpy matplotlib streamlit pandas scikit-learn keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os


zip_file_path = '/content/drive/MyDrive/archive-7.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/chest_xray')

base_dir = '/content/chest_xray/chest_xray'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile


zip_path = '/content/drive/MyDrive/archive-7.zip'
extract_path = '/content/chest_xray'

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)


os.listdir(extract_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['chest_xray']

In [ ]:
train_dir = os.path.join(extract_path, 'chest_xray', 'train')
validation_dir = os.path.join(extract_path, 'chest_xray', 'val')
test_dir = os.path.join(extract_path, 'chest_xray', 'test')

print(os.listdir(train_dir))
print(os.listdir(validation_dir))
print(os.listdir(test_dir))


['NORMAL', 'PNEUMONIA']
['NORMAL', 'PNEUMONIA']
['NORMAL', 'PNEUMONIA']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
     Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()   MaxPooling2D((2, 2)),




Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 17, 17, 128)      

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)


Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 0.1952 - accuracy: 0.9350

100/100 [==============================] - 216s 2s/step - loss: 0.1952 - accuracy: 0.9350 - val_loss: 0.7723 - val_accuracy: 0.6875
Epoch 2/10
100/100 [==============================] - 203s 2s/step - loss: 0.1810 - accuracy: 0.9353
Epoch 3/10
100/100 [==============================] - 203s 2s/step - loss: 0.1663 - accuracy: 0.9409
Epoch 4/10
100/100 [==============================] - 200s 2s/step - loss: 0.1579 - accuracy: 0.9472
Epoch 5/10
100/100 [==============================] - 200s 2s/step - loss: 0.1838 - accuracy: 0.9369
Epoch 6/10
100/100 [==============================] - 197s 2s/step - loss: 0.1620 - accuracy: 0.9406
Epoch 7/10
100/100 [==============================] - 198s 2s/step - loss: 0.1513 - accuracy: 0.9475
Epoch 8/10
100/100 [==============================] - 197s 2s/step - loss: 0.1572 - accuracy: 0.9444
Epoch 9/10
100/100 [==============================] - 199s 2s/step - loss: 0.1417 - accuracy: 0.9522
Epoch 10/10
100/100 [==============================] - 201s 

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")


Found 624 images belonging to 2 classes.
20/20 [==============================] - 15s 745ms/step - loss: 0.4699 - accuracy: 0.8590
Test Accuracy: 0.8589743375778198


In [ ]:
model.save('pneumonia_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
